In [1]:
from sklearn.ensemble import RandomForestClassifier
from transformers import AutoTokenizer, AutoModel
from sklearn.datasets import make_classification
from sklearn.metrics import f1_score
from torch import cuda
import csv
import torch
import sys

device = 'cuda' if cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased').to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

In [2]:
file = open('train.csv')
cr = csv.reader(file)
lines = list(cr)
file.close()

X=[]
Y=[]


In [3]:
for i in lines:
	input_ids = torch.tensor(tokenizer.encode(i[-2])).unsqueeze(0)
	outputs = model(input_ids.to(device))
	last_hidden_states = outputs[0]
	X.append(last_hidden_states.tolist()[0][0])
	Y.append(int(i[-1]))

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X, Y)

RandomForestClassifier(max_depth=2, random_state=0)

In [4]:
file = open('validation.csv')
cr = csv.reader(file)
lines = list(cr)
file.close()

corr = 0

preds = []
Y = []


In [5]:
for i in lines:
	input_ids = torch.tensor(tokenizer.encode(i[-2])).unsqueeze(0)
	outputs = model(input_ids.to(device))
	last_hidden_states = outputs[0]
	pred = clf.predict([last_hidden_states.tolist()[0][0]])
	preds.append(pred)
	Y.append(int(i[-1]))
	if pred == int(i[-1]):
		corr+=1


In [6]:
mf1 = f1_score(Y, preds, average='macro')
acc = corr/len(lines)

output_file_name = f"random_forest_thu1_task2.txt"


file = open(output_file_name,'w')
file.write("Macro F1: {0}\nAccuracy: {1}\n".format(mf1,acc))
file.close()
